# 13. Parallelizing Neural Netowrk Training with TensorFlow

In [1]:
%run -i 'watermark.py'

2020-07-23 13:33:40
----------------------
python		3.6.7
----------------------
numpy		1.16.2
scipy		1.1.0
pandas		0.25.1
matplotlib	3.1.1
imageio		2.5.0
----------------------
ipython		7.8.0
----------------------
sklearn		0.20.4
tensorflow	1.13.1
nltk		3.2.4
----------------------
networkx	2.0


## 13.01. TensorFlow and training performance

### 13.01.01. Performance challenges

### 13.01.02. What is TensorFlow?

![Fig.13.1](https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch13/images/01.png)

### 13.01.03. How we will learn TensorFlow

## 13.02. First steps with TensorFlow

### 13.02.01. Installing TensorFlow

### 13.02.02. Creating ttensors in TensorFlow



In [ ]:
import tensorflow as tf
import numpy as np

np.set_printoptions(precision=3)
a = np.array([1, 2, 3], dtype=np.int32)